In [ ]:
# Load data
test = pd.read_csv('/kaggle/input/playground-series-s5e1/test.csv')
sample_submission = pd.read_csv('/kaggle/input/playground-series-s5e1/sample_submission.csv')

# Feature engineering
test['date'] = pd.to_datetime(test['date'])
test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day
test['month_day'] = test['date'].dt.strftime('%m-%d')
test['day_of_week'] = test['date'].dt.dayofweek
test['is_weekend'] = test['day_of_week'].isin([5, 6]).astype(int)
test['is_holiday'] = test['month_day'].isin(holidays).astype(int)
test['is_holiday_weekend'] = test['is_holiday'] * test['is_weekend']
test['day_of_week_is_weekend'] = test['day_of_week'] * test['is_weekend']
test['country_store'] = test['country'] + "_" + test['store']
test['product_store'] = test['product'] + "_" + test['store']

# Scale numerical features
test[numerical_features] = scaler.transform(test[numerical_features])

# Encode categorical features
encoded_test_features = pd.DataFrame(
    encoder.transform(test[['country_store', 'product_store']]),
    index=test.index
)
encoded_test_features.columns = encoder.get_feature_names_out(['country_store', 'product_store'])

# Combine numerical and encoded categorical features
X_test = pd.concat([test[numerical_features], encoded_test_features], axis=1)

# Apply Polynomial Features
X_test_poly = poly.transform(X_test)

# Predict on test set
y_pred_test_log = model.predict(X_test_poly)
y_pred_test = np.expm1(y_pred_test_log)
y_pred_test = np.where(np.isinf(y_pred_test), 5000, y_pred_test)

# Add predictions to test DataFrame
test['predicted_sales'] = y_pred_test

# Submission csv
sample_submission['num_sold'] = y_pred_test
sample_submission.to_csv('/kaggle/working/submission.csv', index=False)
print("Submission file created: submission.csv")